# Create a web application

In our last exercise, we create the applicatoin to run in Jupyter Notebook. 
To provide the same fidelity to end-users we need to turn this into a web application.

For this we need:

* A layout for the web page: the sensor plot, the current alerts as a table, the model drift plot
* the previously developed functions to get the report data and to calculate model drift

We will use a framework called Plotly Dash (https://plotly.com/dash/), which makes it easy to create web applications for data analysis and machine learning.

## Tasks

1. For each step below, study the code and run it
2. Check that the output matches your expectation
3. Restart the machine simulator with a different configuration, observe how the application output changes

In [1]:
# imports
%load_ext autoreload
%autoreload 

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import jupyter
from dashserve import JupyterDash
from dash.dependencies import Output, Input
import dash_table

from util import load_model, read_data, fix

/tmp/ipykernel_1711024/542329617.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_1711024/542329617.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_1711024/542329617.py:13: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


# Create the basic layout

In [6]:
def create_app():
    fig_sensor = px.scatter()
    fig_drift = px.scatter()

    app = JupyterDash(__name__)
    app.layout = html.Div(children=[
        html.H1(children='Predictive Maintenace App'),
        dcc.Tabs([
            dcc.Tab(label='Machine Status', children=[
                dcc.Graph(id='sensor-graph',
                          figure=fig_sensor)
            ]),
            dcc.Tab(label='Alerts', children=[
                dash_table.DataTable(id='alerts-table',
                                 columns=[dict(name='dt', id='dt'),
                                          dict(name='value', id='value')]),
            ]),
            dcc.Tab(label='Model drift', children=[
                dcc.Graph(id='drift-graph',
                          figure=fig_drift),
            ]), 
        ]),
        dcc.Interval(id='interval-component',
                     interval=1*1000, # in milliseconds
                     n_intervals=0)
    ])
    return app

if __name__ == '__main__':
    app = create_app()
    app.run_server(debug=True)

Stopped
Dash is running on http://localhost:8050/



INFO:__main__:Dash is running on http://localhost:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://localhost:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:16] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:16] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:16] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:16] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:27] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:27] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:27] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:27] "GET 

# Add sensor data

In [7]:
def get_report_data(model, alerts):
    # read the data from the machine API
    df = read_data(100)
    df['alert'] = False
    df['time'] = df.index
    # use the model to predict outliers
    y_hat = model.predict(fix(df['value']))
    # mark all outliers and record to alerts
    df['alert'] = y_hat == -1
    all_alerts = df[df['alert']]
    for i, row in all_alerts.iterrows():
        alerts.update({row.name: row['value']})
    return df, alerts


def add_sensor_plot(app):
    @app.callback(
        Output("sensor-graph", "figure"),
        Input("interval-component", "n_intervals")
    )
    def update_sensor_plot(n_intervals):
        df, _ = get_report_data(model, alerts)
        fig = px.scatter(df, 'time', 'value', color='alert', range_y=(0, 1))
        return fig

app = create_app()
alerts = {}
model = load_model('models/mymodel')
add_sensor_plot(app)
    
if __name__ == '__main__':
    app.run_server(debug=True)

Stopped
Dash is running on http://localhost:8050/



INFO:__main__:Dash is running on http://localhost:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://localhost:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:42] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:43] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:44] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49

# Add alert table

In [8]:
def add_alerts_table(app):
    @app.callback(
        Output("alerts-table", "data"),
        Input("interval-component", "n_intervals")
    )
    def update_alerts_table(n_intervals):
        _, all_alerts = get_report_data(model, alerts)
        df = pd.DataFrame({'dt': all_alerts.keys(), 'value': all_alerts.values()})
        return df.to_dict(orient='records')
    
app = create_app()
add_sensor_plot(app)
add_alerts_table(app)
    
if __name__ == '__main__':
    app.run_server(debug=True)

Stopped
Dash is running on http://localhost:8050/



INFO:__main__:Dash is running on http://localhost:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://localhost:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:53] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:53] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:53] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49:54] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:49

# Add model drift

In [9]:
def calculate_expected_distribution(model, df):
    y_hat = model.predict(fix(df['value']))
    df['alert'] = y_hat == -1
    return df['alert'].value_counts(normalize=True)

def calculate_model_drift(df, expected):
    actual = df['alert'].value_counts(normalize=True)
    df = pd.DataFrame({
         'actual': actual,
         'expected': expected
    })
    return df


def add_drift_plot(app):
    train_data = pd.read_csv('datasets/traindata.csv')    
    model = load_model('models/mymodel')
    expected = calculate_expected_distribution(model, train_data)

    @app.callback(
        Output("drift-graph", "figure"),
        Input("interval-component", "n_intervals")
    )
    def update_drift_plot(n_intervals):
        df, _ = get_report_data(model, alerts)
        df_drift = calculate_model_drift(df, expected)
        fig = px.bar(df_drift, y=['actual', 'expected'], barmode='group')
        return fig

app = create_app()
add_sensor_plot(app)
add_alerts_table(app)
add_drift_plot(app)
    
if __name__ == '__main__':
    app.run_server(debug=True)

Stopped
Dash is running on http://localhost:8050/



INFO:__main__:Dash is running on http://localhost:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://localhost:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:00] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:00] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:00] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:01] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:02] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:02] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:02] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:02] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2021 01:50:02] "POST /_dash-update-component HTTP/1.1" 2